# Notebook 5 : Web scraping

In [ ]:
# Décommenter la ligne suivante pour installer lxml (nécessaire pour read_html)
#!pip install lxml

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
import requests

from bs4 import BeautifulSoup

## Bonsaïs

Le site [Umi Zen Bonsai](https://umizenbonsai.com/) est une boutique de vente en ligne dédiée aux bonsaïs. Les conifères sont disponible sur la page web [https://umizenbonsai.com/shop/bonsai/coniferes/](https://umizenbonsai.com/shop/bonsai/coniferes/). Comme beaucoup d'autres sites, l'information est organisée en blocs dans lesquels il est possible de récupérer des données.

Pour scraper ce type de site, le processus consiste à capturer les blocs dans un premier temps, puis à en extraire les données.

1. Récupérer le contenu de la page avec `requests` et passer le résultat au parser de `BeautifulSoup`.

In [50]:
url_zenbonsai = "https://umizenbonsai.com/shop/bonsai/coniferes/"

zenbonsai = requests.get(url_zenbonsai)
assert zenbonsai.status_code == 200, f"Erreur {zenbonsai.status_code}"

soup_zenbonsai = BeautifulSoup(zenbonsai.text, "html.parser")

2. Écrire un sélecteur CSS pour capturer les éléments `li` qui contiennent les blocs correspondants aux bonsaïs. Vérifier sur le site que le nombre de bonsaïs affichés correspond.

In [51]:
selector_bonsais = ".product-inner"
bonsais = soup_zenbonsai.select(selector_bonsais)

print(len(bonsais))



9


3. Écrire une fonction qui prend un bloc de la liste précédente et retourne un tuple contenant le nom, le prix et le lien de description du bonsaï.

In [ ]:
def f(page):
    soup_zenbonsai = BeautifulSoup(page.text, "html.parser")

    selector_bonsais = ".product-inner"
    bonsais = soup_zenbonsai.select(selector_bonsais)
    
    bonsais_info = []
    for bonsai in bonsais:
        selector_price = ".woocommerce-Price-amount"
        price = bonsai.select(selector_price)[0]

        selector_title = ".title"
        title = bonsai.select(selector_title)[0]
        
        selector_href = ".title h2 a"
        href = bonsai.select(selector_href)[0]
        href_text = href.attrs["href"]

        bonsais_info.append({
            "title": title.text,
            "price": price.text,
            "href": href_text
        })
    return bonsais_info



[{'title': 'Bonsai Cyprès du Japon – 33cm',
  'price': '140.00€',
  'href': 'https://umizenbonsai.com/vente/bonsai-cypres-du-japon/'},
 {'title': 'Bonsai Genévrier de Phénicie – 59cm',
  'price': '250.00€',
  'href': 'https://umizenbonsai.com/vente/bonsai-genevrier-de-phenicie/'},
 {'title': 'Bonsai Genévrier de Phénicie – 67cm',
  'price': '370.00€',
  'href': 'https://umizenbonsai.com/vente/bonsai-genevrier-phenicie/'},
 {'title': 'Bonsai If Commun – 58cm',
  'price': '1,190.00€',
  'href': 'https://umizenbonsai.com/vente/bonsai-if-commun-2/'},
 {'title': 'Bonsai If Commun – 85cm',
  'price': '650.00€',
  'href': 'https://umizenbonsai.com/vente/bonsai-if-commun/'},
 {'title': 'Bonsai If du Japon – 50cm',
  'price': '750.00€',
  'href': 'https://umizenbonsai.com/vente/bonsai-if-du-japon/'},
 {'title': 'Bonsai If du Japon – 63cm',
  'price': '1,290.00€',
  'href': 'https://umizenbonsai.com/vente/bonsai-if-japon/'},
 {'title': 'Bonsai Pin Sylvestre – 38cm',
  'price': '440.00€',
  'href

4. Utiliser les deux questions précédentes pour construire un dataframe contenant les données des bonsaïs.

In [49]:
bonsais_info = f(zenbonsai)

bonsais_df = pd.DataFrame(bonsais_info)


5. (*Bonus*) Écrire une fonction pour récupérer la provenance, le feuilage et les dimension du bonsaï à partir du lien de description. Utiliser cette fonction pour ajouter des colonnes au dataframe précédent

## Trampoline

Le trampoline est un sport olympique depuis les jeux de Sydney en 2000. La page suivante contient les listes des hommes et des femmes ayant obtenu une médaille olympique dans cette discipline :
[https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline](https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline)

Un tableau est contenu dans un élément `table` avec des balises pour les lignes `tr`, pour les colonnes `th`, pour les cellules `td`, ... Cela peut être fastidieux à scraper et très répétitif. Heureusement, Pandas propose la fonction `read_html` pour récupérer des tableaux sous forme de dataframes à partir d'une page web.

1. Utiliser la fonction `read_html` de Pandas sur la page des médaillés olympiques au trampoline. Combien de dataframes sont récupérés ?

2. Extraire de la liste précédente les dataframes des médailles masculines et féminines.

3. À partir de ces dataframes, compter combien chaque pays a reçu de médailles d'or, d'argent et de bronze.

4. (*Bonus*) Construire un dataframe contenant, pour chaque pays, le nombre de médailles d'or, d'argent et de bronze ainsi que le nombre total de médailles. Classer ce dataframe dans l'ordre usuel en fonction d'abord du nombre de médailles d'or, puis du nombre de médailles d'argent et enfin du nombre de médailles de bronze. Comparer le résultat avec le tableau des médailles sur la page [https://fr.wikipedia.org/wiki/Trampoline_aux_Jeux_olympiques](https://fr.wikipedia.org/wiki/Trampoline_aux_Jeux_olympiques).

## Cate Blanchett

Dans le cours, nous avons essayé de trouver avec quels acteurs Cate Blanchett a joué le plus au cours des années 2000. Pour cela, nous avons récupéré la liste des pages Wikipedia des films où elle tient un rôle avec le code suivant :

In [55]:
url_wikipedia = "https://fr.wikipedia.org"
url_blanchett = url_wikipedia + "/wiki/Cate_Blanchett"

r_blanchett = requests.get(url_blanchett)
assert r_blanchett.status_code == 200, f"Erreur {r_blanchett.status_code}"

soup_blanchett = BeautifulSoup(r_blanchett.text, "html.parser")

selector_films = "#mw-content-text div ul:nth-of-type(3) li i a"
films_blanchett = soup_blanchett.select(selector_films)

films_data = [
    {
        "titre": film.attrs["title"],
        "url_wikipedia": url_wikipedia + film.attrs["href"]
    }
    for film in films_blanchett
    if not (
        film.attrs.get("class") == ["new"] # Film sans page
        or film.attrs["title"] == "Galadriel" # Mauvais lien
    )
]

films = pd.DataFrame(films_data)

films

,titre,url_wikipedia
0,Les Larmes d'un homme,https://fr.wikipedia.org/wiki/Les_Larmes_d%27u...
1,Intuitions,https://fr.wikipedia.org/wiki/Intuitions
2,"Bandits (film, 2001)","https://fr.wikipedia.org/wiki/Bandits_(film,_2..."
3,Le Seigneur des anneaux : La Communauté de l'a...,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...
4,Charlotte Gray,https://fr.wikipedia.org/wiki/Charlotte_Gray
5,Terre Neuve (film),https://fr.wikipedia.org/wiki/Terre_Neuve_(film)
6,"Heaven (film, 2002)","https://fr.wikipedia.org/wiki/Heaven_(film,_2002)"
7,Le Seigneur des anneaux : Les Deux Tours,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...
8,Veronica Guerin (film),https://fr.wikipedia.org/wiki/Veronica_Guerin_...
9,Coffee and Cigarettes,https://fr.wikipedia.org/wiki/Coffee_and_Cigar...


Le sélecteur CSS que nous avons utilisé ne permettait pas d'obtenir la réponse à notre question car il ne capturait pas toutes les listes d'acteurs (organisation différente pour *Coffee and Cigarettes*, double colonne pour *Aviator*, ...). En effet, les pages Wikipedia des films ne sont pas uniformes et il n'est pas possible d'extraire la distribution de tous les films avec le même sélecteur.

Pour remédier à cela, nous proposons ici d'aller scraper la liste des acteurs sur le site [TMDB](https://www.themoviedb.org/) (*The Movie Database*) dont les pages obéissent toutes à la même organisation. Les pages Wikipedia relatives à des films contiennent toutes un lien externe vers ce site.

1. Pour chaque film, scraper la page Wikipedia pour récupérer le lien vers la page TMDB associée et déduire le lien du casting complet qui ser ajouté le dans une nouvelle colonne du dataframe `films`.

In [83]:
# Créer une liste pour stocker les URL de casting
url_casting_list = []

for index, film in films.iterrows():

    url = film["url_wikipedia"]  # Récupérer l'URL Wikipedia du film
    
    html = requests.get(url)
    assert html.status_code == 200, f"Erreur {html.status_code}"

    soup = BeautifulSoup(html.text, "html.parser")

    selector_url_externes = "a.external"
    url_externes = soup.select(selector_url_externes)

    url_TMDB = ""

    for url_externe in url_externes:
        if url_externe.text == "The Movie Database":
            print("The Movie Database URL found!")
            url_TMDB = url_externe.attrs["href"]
            # Vous pouvez ajouter du code ici pour traiter l'URL trouvée
            # Par exemple, vous pourriez l'ajouter à un DataFrame ou l'enregistrer dans un fichier
            break
    
    print(url_TMDB)
    url_casting_list.append(url_TMDB+"/cast")  # Ajouter l'URL à la liste

    """
    html2 = requests.get(url_TMDB)

    html2 = requests.get(url)
    assert html2.status_code == 200, f"Erreur {html2.status_code}"

    soup2 = BeautifulSoup(html2.text, "html.parser")

    print(soup2)
    """
    
    #break
films["url_casting"] = url_casting_list

The Movie Database URL found!
https://www.themoviedb.org/movie/29572
The Movie Database URL found!
https://www.themoviedb.org/movie/2046
The Movie Database URL found!
https://www.themoviedb.org/movie/3172
The Movie Database URL found!
https://www.themoviedb.org/movie/120
The Movie Database URL found!
https://www.themoviedb.org/movie/12660
The Movie Database URL found!
https://www.themoviedb.org/movie/6440
The Movie Database URL found!
https://www.themoviedb.org/movie/10575
The Movie Database URL found!
https://www.themoviedb.org/movie/121
The Movie Database URL found!
https://www.themoviedb.org/movie/10629
The Movie Database URL found!
https://www.themoviedb.org/movie/883
The Movie Database URL found!
https://www.themoviedb.org/movie/12146
The Movie Database URL found!
https://www.themoviedb.org/movie/122
The Movie Database URL found!
https://www.themoviedb.org/movie/421
The Movie Database URL found!
https://www.themoviedb.org/movie/2567
The Movie Database URL found!
https://www.themov

2. La liste des acteurs d'un film se présente comme une liste ordonnée `ol` dans les pages TMDB. Scraper les pages de casting pour ajouter la liste des acteurs de chaque film dans une nouvelle colonne du dataframe `films`.

In [ ]:
films

for index, film in films.iterrows():
    
    url = film["url_casting"]
    
    html = requests.get(url)
    assert html.status_code == 200, f"Erreur {html.status_code}"
    #print(url)
    soup = BeautifulSoup(html.text, "html.parser")
    selector_ol_people = "ol.people.credits li"
    # Fonctionne :!!!
    # selector_ol_people = "section > ol.people.credits li"
    ol_peoples = soup.select(selector_ol_people)
    print(len(ol_peoples))
    print(ol_peoples)

    for ol_people in ol_peoples:
        #print("###################")
        #print(ol_people)

        selector_li = "div.info p a"
        li = ol_people.select(selector_li)
        #print(li)
        print(li[0].text)

    break
    

97
[<li data-credit-id="52fe45eac3a368484e0764f3" data-order="0">
<a href="/person/6886-christina-ricci">
<div class="glyphicons_v2 picture grey profile no_image_holder one">
<img alt="Christina Ricci" class="profile w-full" loading="lazy" src="https://media.themoviedb.org/t/p/w66_and_h66_face/dzB58d6fNrTEi7nBAU1tySJc2at.jpg" srcset="https://media.themoviedb.org/t/p/w66_and_h66_face/dzB58d6fNrTEi7nBAU1tySJc2at.jpg 1x, https://media.themoviedb.org/t/p/w132_and_h132_face/dzB58d6fNrTEi7nBAU1tySJc2at.jpg 2x"/>
</div>
</a>
<div>
<div class="info">
<p><a href="/person/6886-christina-ricci">Christina Ricci</a><p>
<p class="character">Suzie
            </p>
</p></p></div>
</div>
</li>, <li data-credit-id="52fe45eac3a368484e0764f7" data-order="1">
<a href="/person/85-johnny-depp">
<div class="glyphicons_v2 picture grey profile no_image_holder two">
<img alt="Johnny Depp" class="profile w-full" loading="lazy" src="https://media.themoviedb.org/t/p/w66_and_h66_face/wcI594cwM4ArPwvRd2IU0Z0yLuh.jpg"

3. Utiliser le résultat de la question précédente pour répondre à la question initiale : avec quels acteurs Cate Blanchett a-t-elle partagé l'affiche le plus souvent au cours des années 2000 ?